# Classification de texte avec MLP et embeddings SpaCy

Ce notebook détaille un pipeline complet de classification de texte utilisant un réseau de neurones (MLP) et les embeddings pré-entraînés de SpaCy. Chaque étape est expliquée, du prétraitement à la génération du fichier de soumission.

---

## Concepts clés

- **Embeddings SpaCy** : chaque mot est représenté par un vecteur dense issu d'un modèle pré-entraîné, capturant la sémantique du mot.
- **Moyenne des embeddings** : pour chaque texte, on fait la moyenne des vecteurs des mots pour obtenir une représentation globale.
- **MLPClassifier** : réseau de neurones à couches entièrement connectées (Multi-Layer Perceptron) pour la classification.
- **Prétraitement linguistique** : nettoyage, lemmatisation, suppression des stopwords pour améliorer la qualité des features.

---

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import re
import spacy
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import numpy as np
import json

## Version et introduction

On affiche la version du script et on explique le but : entraîner un MLP sur des embeddings SpaCy pour la classification de texte.

In [ ]:
version_ = "V.4.0.2"
print("Version:", version_)

## Chargement du modèle SpaCy pré-entraîné

On utilise le modèle SpaCy pour lemmatiser et vectoriser les mots. Les embeddings sont appris sur de grands corpus et capturent la similarité sémantique entre mots.

In [ ]:
print("Loading pre-trained SpaCy model...")
nlp = spacy.load('en_core_web_sm')

## Chargement et nettoyage des données d'entraînement

On charge les données d'entraînement et on remplace les valeurs manquantes dans la colonne 'description' par une chaîne vide.

In [ ]:
print("Loading training data...")
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data)
print("Checking for problematic rows in 'description'...")
df['description'] = df['description'].fillna("")

## Nettoyage et lemmatisation des textes

On définit une fonction qui :
- enlève le HTML et les URLs
- met en minuscules
- lemmatise et enlève les stopwords et tokens non alphabétiques

On retourne à la fois le texte nettoyé et la liste des tokens pour la vectorisation.

In [ ]:
def clean_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return "", []
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = text.lower().strip()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens), tokens

On applique la fonction à chaque description pour obtenir deux colonnes :
- `Clean` : texte nettoyé et lemmatisé
- `Tokens` : liste des tokens pour la vectorisation

In [ ]:
print("Cleaning and tokenizing training data...")
results = df['description'].apply(clean_and_tokenize)
tuples = results.apply(lambda x: x if (isinstance(x, tuple) and len(x) == 2) else ("", []))
results_df = pd.DataFrame(tuples.tolist(), index=results.index, columns=['Clean', 'Tokens'])
df[['Clean', 'Tokens']] = results_df

## Chargement des labels

On associe à chaque texte sa catégorie cible pour l'apprentissage supervisé.

In [ ]:
print("Loading labels...")
labels_df = pd.read_csv('train_label.csv')
df = df.merge(labels_df, on='Id')

## Vectorisation des textes par moyenne des embeddings SpaCy

Pour chaque texte, on calcule la moyenne des vecteurs SpaCy de ses mots. Cela donne un vecteur de taille fixe représentant le texte entier.

Si un mot n'a pas de vecteur, il est ignoré. Si aucun mot n'est connu, on retourne un vecteur nul.

In [ ]:
def vectorize_tokens_spacy(tokens, nlp_model):
    vectors = [nlp_model(token).vector for token in tokens if token.strip()]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(nlp_model("word").vector.shape[0])

print("Vectorizing tokens using SpaCy's pre-trained model...")
df['Vector'] = df['Tokens'].apply(lambda tokens: vectorize_tokens_spacy(tokens, nlp))

## Préparation des données pour l'apprentissage supervisé

On construit la matrice X (features) et le vecteur y (labels) pour entraîner le modèle de classification.

In [ ]:
print("Preparing training data...")
X = np.vstack(df['Vector'].values)
y = df['Category'].values

## Entraînement du modèle de réseau de neurones (MLP)

On utilise un MLP (Multi-Layer Perceptron) avec deux couches cachées. Le MLP apprend à séparer les classes dans l'espace vectoriel des textes.

Mathématiquement, le MLP apprend une fonction $f(x) = W_2 \cdot \text{ReLU}(W_1 x + b_1) + b_2$ où $x$ est le vecteur du texte.

In [ ]:
print("Training Neural Network model...")
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
nn_model.fit(X, y)

## Chargement et prétraitement des données de test

On applique le même nettoyage et la même vectorisation aux textes de test.

In [ ]:
print("Loading test data...")
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data)

print("Cleaning and tokenizing test data...")
test_df[['Clean', 'Tokens']] = test_df['description'].apply(lambda t: pd.Series(clean_and_tokenize(t)))

## Vectorisation des textes de test

On calcule la moyenne des embeddings SpaCy pour chaque texte de test.

In [ ]:
print("Vectorizing test data using SpaCy's pre-trained model...")
test_df['Vector'] = test_df['Tokens'].apply(lambda tokens: vectorize_tokens_spacy(tokens, nlp))
X_test = np.vstack(test_df['Vector'].values)

## Prédiction des catégories sur les données de test

Le modèle prédit la catégorie la plus probable pour chaque texte de test.

In [ ]:
print("Predicting categories for test data...")
test_df['Predicted_Category_NN'] = nn_model.predict(X_test)

## Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
print("Preparing submission file...")
template = pd.read_csv('template_submissions.csv')
template['Category'] = template['Id'].map(
    test_df.set_index('Id')['Predicted_Category_NN']
)
template.to_csv('submission4.csv', index=False)
print("Submission file saved as 'submission4.csv'.")
print("Version:", version_)

---

## Concepts mathématiques et conclusion

- **Embeddings SpaCy** : chaque mot est représenté par un vecteur dense appris sur de grands corpus. La moyenne des embeddings donne une représentation globale du texte.
- **MLP** : le réseau apprend une fonction non linéaire pour séparer les classes dans l'espace vectoriel.
- **Classification** : la sortie du MLP est une probabilité pour chaque classe, la classe avec la plus grande probabilité est choisie.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un réseau de neurones, en utilisant des représentations vectorielles avancées.